In [1]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

'/Users/ptrbelonovsky/Desktop/DSBA4YEAR/KRIS/RecSys_thesis/recsysvenv/bin/python3.12'

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# from scipy.sparse import csr_matrix
# from scipy.sparse.linalg import svds
# from surprise import SVDpp, Reader, Dataset
# from surprise.model_selection import cross_validate

from typing import Union
from tqdm.notebook import tqdm

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch

from src.metrics import reccomendation_report
from src.utils import surprise_predict
from src.utils import split_test_df
from src.utils import seed_everything
from src.utils import trainDataset
from src.utils import trainDatasetWithNumCatFeatures

from src.models import DeepFM

import mmh3

## Constants

In [12]:
DATA_FOLDER = "../../data/ml-1m/"
RANDOM_STATE = 7

In [13]:
np.random.seed(RANDOM_STATE)

## Data

In [14]:
df_movies = pd.read_csv(
    DATA_FOLDER + "movies.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["movieId", "name", "genre"],
)
df_ratings = pd.read_csv(
    DATA_FOLDER + "ratings.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["userId", "movieId", "rating", "timestamp"],
)
df_users = pd.read_csv(
    DATA_FOLDER + "users.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["userId", "gender", "age", "occupation", "zip-code"],
)

In [15]:
## Encode usedId, movieId
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

df_movies["movieId"] = movie_encoder.fit_transform(df_movies["movieId"])
df_users["userId"] = user_encoder.fit_transform(df_users["userId"])

df_ratings["movieId"] = movie_encoder.transform(df_ratings["movieId"])
df_ratings["userId"] = user_encoder.transform(df_ratings["userId"])

### Train-test split
Methodology: Last user interaction is a test item. The rest is train. Validation part is 10% of test.

In [16]:
df_ratings["rank"] = (
    df_ratings[["userId", "timestamp"]]
    .groupby("userId", as_index=False)["timestamp"]
    .rank(method="first", ascending=False)
)
# df_ratings = df_ratings.merge(
#     pd.DataFrame(df_ratings["userId"].value_counts()).reset_index(),
#     how="left", on="userId")
# df_ratings["cum_position"] = df_ratings["rank"] / df_ratings["count"]
# df_ratings = df_ratings.drop(columns=["rank", "count"])

In [17]:
# leave one out
df_train = df_ratings.loc[df_ratings["rank"] != 1].reset_index(drop=True)
df_test = (
    df_ratings.loc[df_ratings["rank"] == 1].reset_index(drop=True).assign(action=1)
)
df_test, df_val = train_test_split(df_test, test_size=0.2, random_state=RANDOM_STATE)

In [18]:
# enrich test data with 100 random movies from the ones not intercated by user
df_add = pd.DataFrame()
for user in tqdm(df_test.userId.unique(), desc="Enriching test"):
    movie = df_test.loc[df_test.userId == user, "movieId"]
    watched_movies = np.append(
        movie, df_train.loc[df_train.userId == user, "movieId"].values
    )
    not_wathed_movies = np.setdiff1d(
        np.arange(df_movies["movieId"].max() + 1), watched_movies
    )
    random_100 = np.random.choice(not_wathed_movies, 100, replace=False)

    df_temp = pd.DataFrame().assign(movieId=random_100, userId=user, action=0)
    df_add = pd.concat([df_add, df_temp], ignore_index=True)

df_test = pd.concat([df_test, df_add], ignore_index=True).drop(
    columns=["timestamp", "rating", "rank"]
)

df_add = pd.DataFrame()
for user in tqdm(df_val.userId.unique(), desc="Enriching val"):
    movie = df_val.loc[df_val.userId == user, "movieId"]
    watched_movies = np.append(
        movie, df_train.loc[df_train.userId == user, "movieId"].values
    )
    not_wathed_movies = np.setdiff1d(
        np.arange(df_movies["movieId"].max() + 1), watched_movies
    )
    random_100 = np.random.choice(not_wathed_movies, 100, replace=False)

    df_temp = pd.DataFrame().assign(movieId=random_100, userId=user, action=0)
    df_add = pd.concat([df_add, df_temp], ignore_index=True)

df_val = pd.concat([df_val, df_add], ignore_index=True).drop(
    columns=["timestamp", "rating", "rank"]
)

Enriching test:   0%|          | 0/4832 [00:00<?, ?it/s]

Enriching val:   0%|          | 0/1208 [00:00<?, ?it/s]

## Building one-hot-encoded featurues and normalizing continious features

In [30]:
ord_user = OrdinalEncoder()
user_cat = torch.tensor(ord_user.fit_transform(df_users[["gender", "occupation"]]))

ord_movie = OrdinalEncoder()
movie_cat = torch.tensor(ord_movie.fit_transform(df_movies[["genre"]]))

ss_user = StandardScaler()
user_num = torch.tensor(ss_user.fit_transform(df_users[["age"]]))

In [20]:
# train_loader = DataLoader(
#     trainDatasetWithNumCatFeatures(
#         df_train,
#         df_movies["movieId"].nunique(),
#         user_cat,
#         user_num,
#         movie_cat
#     ),
#     batch_size=2048,
#     shuffle=True,
# )

In [34]:
test_user_cat = user_cat[df_test["userId"].values].clone().detach().to(torch.long)
test_user_num = user_num[df_test["userId"].values].clone().detach().to(torch.float)
test_movie_cat = movie_cat[df_test["movieId"].values].clone().detach().to(torch.long)
test_cat = torch.hstack((test_user_cat, test_movie_cat))

test_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test["userId"]),
        torch.tensor(df_test["movieId"]),
        test_user_num,
        test_cat,
    ),
    batch_size=4096,
    shuffle=False,
)

In [16]:
# import pickle
# with open('data/train_loader.pkl', "wb") as f:
#     pickle.dump(train_loader, f)

In [1]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# from scipy.sparse import csr_matrix
# from scipy.sparse.linalg import svds
# from surprise import SVDpp, Reader, Dataset
# from surprise.model_selection import cross_validate

from typing import Union
from tqdm.notebook import tqdm

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch

from src.metrics import reccomendation_report
from src.utils import surprise_predict
from src.utils import split_test_df
from src.utils import seed_everything
from src.utils import trainDataset
from src.utils import trainDatasetWithNumCatFeatures

from src.models import DeepFM

import mmh3

In [2]:
import pickle

with open("data/train_loader.pkl", "rb") as f:
    train_loader = pickle.load(f)

### DeepFM

In [38]:
num_users = 6040  # df_users["userId"].nunique()
num_items = 3883  # df_movies["movieId"].nunique()
num_numeric_feats = 1
cat_feature_vocab = [
    2,
    21,
    301,
]  # [len(i) for i in ord_user.categories_] + [len(i) for i in ord_movie.categories_]

In [39]:
dF = DeepFM(num_users, num_items, num_numeric_feats, cat_feature_vocab)
display(dF)

DeepFM(
  (user_embedding): Embedding(6040, 5)
  (item_embedding): Embedding(3883, 5)
  (numerical_embeddings): ModuleList(
    (0): Linear(in_features=1, out_features=5, bias=True)
  )
  (categorical_embeddings): ModuleList(
    (0): Embedding(2, 5)
    (1): Embedding(21, 5)
    (2): Embedding(301, 5)
  )
  (mlp): Sequential(
    (MLP_layer_0): Linear(in_features=30, out_features=16, bias=True)
    (Activation_layer_0): ReLU()
    (MLP_layer_1): Linear(in_features=16, out_features=32, bias=True)
    (Activation_layer_1): ReLU()
    (MLP_layer_2): Linear(in_features=32, out_features=64, bias=True)
    (Activation_layer_2): ReLU()
    (MLP_layer_3): Linear(in_features=64, out_features=1, bias=True)
    (Activation_layer_3): ReLU()
  )
  (fm_linear): Linear(in_features=325, out_features=1, bias=True)
  (final_sigmoid): Sigmoid()
)

In [40]:
optimizer = torch.optim.Adam(dF.parameters(), lr=3e-4)
criterion = nn.BCELoss()
n_epochs = 1
device = "cpu"

In [41]:
### Train DeepFM
dF.to(device)
num_iterations = len(train_loader)

for epoch in tqdm(range(n_epochs), desc="Epochs"):
    # train
    total_train_loss = 0
    dF.train()
    with tqdm(train_loader, unit="batch") as tepoch:
        for userIds, movieIds, num_feats, cat_feats, ratings in tepoch:
            pred_train = dF(
                userIds.to(device),
                movieIds.to(device),
                num_feats.to(device),
                cat_feats.to(device),
            )
            loss_train = criterion(pred_train.flatten(), ratings.to(device))

            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()
            total_train_loss += loss_train.item()
            tepoch.set_postfix(loss=loss_train.item())
    print("Epoch:", epoch)
    print("Train loss", round(total_train_loss / num_iterations, 5))

Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2428 [00:00<?, ?batch/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1061a2ba0>>
Traceback (most recent call last):
  File "/Users/ptrbelonovsky/Desktop/DSBA4YEAR/KRIS/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt



In [35]:
# predict dF
dF.eval()
total_preds = torch.zeros(len(test_loader.dataset))
batch_size = test_loader.batch_size
for i, (userIds, movieIds, num_feats, cat_feats) in enumerate(
    tqdm(test_loader, desc="Inference", unit="batch")
):
    with torch.no_grad():
        total_preds[i * batch_size : (i + 1) * batch_size] = dF(
            userIds.to(device),
            movieIds.to(device),
            num_feats.to(device),
            cat_feats.to(device),
        ).flatten()

Inference:   0%|          | 0/120 [00:00<?, ?batch/s]

In [36]:
df_test["rating_pred"] = total_preds.numpy()
pred, target = split_test_df(df_test)

In [37]:
reccomendation_report(pred, target, k=50)

{'Hit rate @ K': tensor(0.6329), 'NDCG @ K': tensor(0.1798)}